In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('max_rows', 7)
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv', index_col=0)
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv', index_col=0)
submission_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv', index_col=0)
train_df

In [ ]:
train_df.describe().T

In [ ]:
train_df.info()

In [ ]:
X = train_df
y = X.pop('loss')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

total_df = pd.concat([X, test_df])

scaler = MinMaxScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(total_df))

X_scaled = scaled_features[:X.shape[0]]
test_scaled = scaled_features[X.shape[0]:]

X_scaled.shape, test_scaled.shape, y.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from warnings import simplefilter
simplefilter = True

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[100]),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='relu')
])

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=lr_schedule),
    loss='mae',
    metrics=['accuracy']
)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y)

early_stop = EarlyStopping(min_delta=0.01, patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=2048,
    epochs=100,
    callbacks=[early_stop],
    verbose=0
)


In [ ]:
y_pred = model.predict(test_scaled)

In [ ]:
submission = pd.DataFrame(y_pred, columns=['loss'], index=test_scaled.index).rename_axis('id', axis='rows')
print(submission.head())
submission.to_csv('submission.csv')